In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [2]:
weather_csv_file = "output_data/cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Arlit,61,NE,1632287522,17,18.7369,7.3853,86.45,9.33
1,Hithadhoo,98,MV,1632287522,76,-0.6000,73.0833,82.90,15.05
2,Rikitea,0,PF,1632287523,72,-23.1203,-134.9692,73.85,15.32
3,Chokurdakh,97,RU,1632287523,54,70.6333,147.9167,38.93,9.28
4,Imbituba,100,BR,1632287524,75,-28.2400,-48.6703,58.89,12.35


In [3]:
gmaps.configure(api_key=g_key) 
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig                              

Figure(layout=FigureLayout(height='420px'))

In [4]:
filtered_weather_df = weather_df
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,Kiryat Gat,0,IL,1632287529,100,31.6100,34.7642,78.40,4.90
113,Esperance,0,AU,1632287567,35,-33.8667,121.9000,74.14,4.25
167,Saint-Pierre,0,RE,1632287589,58,-21.3393,55.4781,74.88,2.30
175,Jalu,0,LY,1632287592,30,29.0331,21.5482,79.36,6.98
176,Zhangjiakou,0,CN,1632287592,42,40.8100,114.8794,70.65,2.48
259,Houma,0,US,1632287627,90,29.5958,-90.7195,78.80,5.32
277,Oruzgan,0,AF,1632287412,9,32.8333,66.0000,77.54,1.03
408,Dalby,0,AU,1632287690,11,-27.1833,151.2667,74.98,3.00
411,Chengde,0,CN,1632287691,54,40.9725,117.9361,71.74,5.39
461,San Quintín,0,MX,1632287714,75,30.4833,-115.9500,72.99,7.07


In [15]:
hotel_df = filtered_weather_df
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 18: Kiryat Gat.
Missing field/result... skipping.
------------
Retrieving Results for Index 113: Esperance.
Missing field/result... skipping.
------------
Retrieving Results for Index 167: Saint-Pierre.
Missing field/result... skipping.
------------
Retrieving Results for Index 175: Jalu.
Missing field/result... skipping.
------------
Retrieving Results for Index 176: Zhangjiakou.
Missing field/result... skipping.
------------
Retrieving Results for Index 259: Houma.
Missing field/result... skipping.
------------
Retrieving Results for Index 277: Oruzgan.
Missing field/result... skipping.
------------
Retrieving Results for Index 408: Dalby.
Missing field/result... skipping.
------------
Retrieving Results for Index 411: Chengde.
Missing field/result... skipping.
------------
Retrieving Results for Index 461: San Quintín.
Missing field/result... skipping.
------------
Retrieving Results for Index 496: Mersin Province.
Missing field/result... skipping.
-----

In [14]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,Kiryat Gat,0,IL,1632287529,100,31.6100,34.7642,78.40,4.90
113,Esperance,0,AU,1632287567,35,-33.8667,121.9000,74.14,4.25
167,Saint-Pierre,0,RE,1632287589,58,-21.3393,55.4781,74.88,2.30
175,Jalu,0,LY,1632287592,30,29.0331,21.5482,79.36,6.98
176,Zhangjiakou,0,CN,1632287592,42,40.8100,114.8794,70.65,2.48
259,Houma,0,US,1632287627,90,29.5958,-90.7195,78.80,5.32
277,Oruzgan,0,AF,1632287412,9,32.8333,66.0000,77.54,1.03
408,Dalby,0,AU,1632287690,11,-27.1833,151.2667,74.98,3.00
411,Chengde,0,CN,1632287691,54,40.9725,117.9361,71.74,5.39
461,San Quintín,0,MX,1632287714,75,30.4833,-115.9500,72.99,7.07


In [12]:
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

KeyError: 'Hotel Name'